## Importing all the dependencies

In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
import nltk
from bs4 import BeautifulSoup
import urllib
from nltk.tokenize import sent_tokenize
from nltk.corpus import cmudict
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
import syllables

## Importing the ciklist into Pandas dataframe and doing some preliminary analysis

In [2]:
df = pd.read_excel(r'D:\nlp_internship\Data Science\cik_list.xlsx')

In [3]:
df.shape

(152, 6)

In [4]:
df.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt


In [5]:
df.describe()

,CIK,FYRMO
count,152.000000,152.000000
mean,5861.605263,200220.750000
std,2310.633436,429.991117
min,3662.000000,199402.000000
25%,4447.000000,199906.000000
50%,5907.000000,200101.500000
75%,6201.000000,200605.500000
max,12239.000000,201407.000000


## We need to extract the files from the web, hence hence lets add the hyperlink and extract all the text data.

In [6]:
# we need to extract the files from the web hence hence lets add the hyperlink and ectract all the text data
hyperlink = 'https://www.sec.gov/Archives/'
df['SECFNAME'] = hyperlink + df['SECFNAME']

### Importing stopwords, positive words and negative words from the list

In [28]:
stop_words = pd.read_csv(r'D:\nlp_internship\Data Science\stop_words\StopWords_GenericLong.txt',header=None) # importing stop words
stopword_list = stop_words[0].to_list()
master_dict = pd.read_csv(r'D:\nlp_internship\Data Science\stop_words\LoughranMcDonald_MasterDictionary_2018.csv') # importing the master dict
negative_words = master_dict[master_dict['Negative'] == 2009]['Word']
negative_dict =[word.lower() for word in negative_words if word not in stop_words] # creating negative words dictonary
positive_words = master_dict[master_dict['Positive'] == 2009]['Word']
positive_dict = [word.lower() for word in positive_words if word not in stop_words] # creating positive words dictonary

### Now all the dependencies are defined and all files are imported to proceed with the analysis. I saw that there are nearly 152 documents in the structure. Instead of downloading each file storing it in my SSD and loading it into my ram, and perform  analysis the text I decided to use, urllib library which allows me to load data from any webpage and store it in RAM. which is fast and efficient.

In [49]:
def preliminary_cleaner(text):
    '''
    This function is used to remove all the 
    numerical data in the files. And next line in the file
    '''
    if text:
        array = []
        soup =text
        soup = re.sub('[\d%/$]','',str(soup))
        soup = re.sub("\\\\n",'',soup)
        soup = ' '.join(soup.split())
        return soup
        
    else:
        return None

### Now what I did is created a regular expression for all the three given titles. And then scraped the website for this data and stored in three python arrays. Later I merged the three arrays into a pandas data frame. Which is a highly efficient data-structure for processing the text files.

In [ ]:
array_one,array_two,array_three = [],[],[] # creating arrays
# regular expression for the three components.
mgt_disc_ana = r"item[^a-zA-Z\n]*\d\s*\.\s*management\'s discussion and analysis.*?^\s*item[^a-zA-Z\n]*\d\s*\.*"
qlty_qnt_disc = r"item[^a-zA-Z\n]*\d[a-z]?\.?\s*Quantitative and Qualitative Disclosures about Market Risk.*?^\s*item\s*\d\s*"
rsk_fct = r"item[^a-zA-Z\n]*\d[a-z]?\.?\s*Risk Factors.*?^\s*item\s*\d\s*"
for hyperlink in df['SECFNAME']:
    text = urllib.request.urlopen(hyperlink).read().decode('utf-8') # eastiblishint the client server connection and downloading the data
    # into ram.
    matches_mda = re.findall(mgt_disc_ana, text, re.IGNORECASE | re.DOTALL | re.MULTILINE)
    matches_qlty_qnt_disc= re.findall(qlty_qnt_disc,text,re.IGNORECASE | re.DOTALL | re.MULTILINE)
    matches_rsk_fct = re.findall(rsk_fct,text,re.IGNORECASE | re.DOTALL | re.MULTILINE) 
    array_one.append(preliminary_cleaner(matches_mda))
    array_two.append(preliminary_cleaner(matches_qlty_qnt_disc))
    array_three.append(preliminary_cleaner(matches_rsk_fct))

# Merging all three arrays and storing it as csv for later analysis.

In [ ]:
df1,df2,df3 = pd.DataFrame(array_one),pd.DataFrame(array_two),pd.DataFrame(array_three) #
merged_dataframe = pd.concat([df1,df2,df3],axis = 1)
merged_dataframe.columns = ['matches_mda','matches_qlty_qnt_disc','matches_rsk_fct']
merged_dataframe.to_csv('merged_dataframe_all.csv')

In [29]:
df2 = pd.read_csv(r'D:\nlp_internship\submission\merged_dataframe_all.csv') # loading the merged dataframe

# concatenating the main data frame and newly merged data frame.

In [30]:
final_dataframe = pd.concat([df,df2],axis = 1)
final_dataframe.drop(columns='Unnamed: 0',inplace = True)
final_dataframe.fillna(' ',inplace = True)

In [31]:
final_dataframe.columns

Index(['CIK', 'CONAME', 'FYRMO', 'FDATE', 'FORM', 'SECFNAME', 'matches_mda',
       'matches_qlty_qnt_disc', 'matches_rsk_fct'],
      dtype='object')

In [50]:
def cleaning_stopwords(columns):
    """This function is used to tokenize and removing everything except alphabets in the text files."""
    columns = columns.apply(lambda x : re.sub("[^a-zA-Z]"," ",str(x))) # removing everything except letters
    columns = columns.apply(lambda x : wordpunct_tokenize(x)) # tokenizing the frame
    # cleaning stopwords
    columns = columns.apply(lambda x : [word.lower() for word in x if word not in stopword_list])
    return columns


# Creating a new data frame called the final data frame which consists of all the tokenized documents.

In [34]:
final_dataframe['mda_bulk'] = cleaning_stopwords(final_dataframe['matches_mda'])
final_dataframe['qltqnt_bulk'] = cleaning_stopwords(final_dataframe['matches_qlty_qnt_disc'])
final_dataframe['rskfct_bulk'] = cleaning_stopwords(final_dataframe['matches_rsk_fct'])


### I have tried to compute the required variables using three methods this method is highly efficient and calculates all the variables in the number of minutes because I am creating a class called text analysis which will initialize the four variables words,complex_words,positive_words and negative words. hence for each of the three sections, I will create a new object for that section and compute all the 14 concerned variables.

In [53]:
class text_analysis:
    def __init__(self,column_with_words,column_with_sentences):
        self.column_with_words = column_with_words # initializing the column which contaions words.
        self.column_with_sentences = column_with_sentences # initializing the words which contains sentence tokens.
        self.words = self.column_with_words.apply(lambda x : len([word for word in x])) # initialing the variable wors count
        self.complex_words = self.column_with_words.apply(lambda x : len([word for word in x if syllables.estimate(word) > 2])) # initializing the variable complex words
        self.positive_words_column = self.column_with_words.apply(lambda x : len([word for word in x if word in np.array(positive_dict)])) # initializing the column number of positive words
        self.negative_words_column = self.column_with_words.apply(lambda x : len([word for word in x if word in np.array(negative_dict)])) # initializing the column number of positive words
    def positive_score(self):
        '''This score is calculated by assigning the value of +1 for each word if 
        found in the Positive Dictionary and then adding up all the values. '''
        return self.positive_words_column
        #return self.column_with_words.apply(lambda x : len([word for word in x if word in positive_dict]))
    def negative_score(self):
        '''This score is calculated by assigning the value of +1 for each word if 
        found in the Negative Dictionary and then adding up all the values.'''
        return self.negative_words_column
        #return self.column_with_words.apply(lambda x : len([word for word in x if word in negative_dict]))
    def polarity_score(self):
        """This is the score that determines if a given text is positive or negative in nature. It is calculated by using the formula: 
            Polarity Score = (Positive Score – Negative Score)/ ((Positive Score + Negative Score) + 0.000001)
        """
        val1 = (self.positive_words_column - self.negative_words_column)
        val2 = (self.positive_words_column + self.negative_words_column + 0.00001)
        return val1 / val2
    def average_sentence_length(self):
        """
        Average Sentence Length = the number of words / the number of sentences
        we use sent_tokenizer liabrary and we replace null values with 0

        """
        #words = self.column_with_words.apply(lambda x : len([word for word in x]))
        sentences = self.column_with_sentences.apply(lambda x : len(sent_tokenize(str(x))))
        ans =  self.words / sentences
        ans = ans.fillna(0)
        return ans
    def syllables_count(self,word):
        """This function is used to count the syllables. In this case, we use the cmu dictionary which consists 
        of all the syllables of given words but I found that there are some words which are not defined in the 
        cmu dictionary hence I used the algorithm to calculate the number of syllables."""
        phoneme_dictonary = dict(cmudict.entries())
        count = 0
        vowels = 'aeiouy'
        word = word.lower()
        if word[0] in vowels:
            count += 1
        for i in range(1,len(word)):
            if word[i] in vowels and word[i - 1] not in vowels:
                count += 1
        if word[-1] == 'e':
            count -= 1
        if word[-2:] == 'le':
            count += 1
        if count == 0:
            count += 1
        # comparing the values with cmudict
        try:
            second_count = 0
            for val in phoneme_dictonary[word.lower()]:
                for val1 in val:
                    if val1[-1].isdigit():
                        second_count += len(val1)
            return second_count
        except KeyError: # this is in case the values are not found in the cmu dict
            return count
    def complex_words_proportion(self):
        """Percentage of Complex words = the number of complex words / the number of words """
        #complex_words = self.column_with_words.apply(lambda x : len([word for word in x if self.syllables_count(word) > 2]))
        #words = self.column_with_words.apply(lambda x : len([word for word in x]))
        ans = self.complex_words / self.words
        ans = ans.fillna(0)
        return ans

    def fog_index(self):
        """Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)
        The Gunning Fog Index gives the number of years of education that your reader 
        hypothetically needs to understand the paragraph or text. The Gunning Fog Index 
        formula implies that short sentences written in plain English achieve a better 
        score than long sentences written in complicated language."""
        return 0.4 * (self.average_sentence_length() + self.complex_words)

    def complex_words_count(self):
        """ Complex words are words in the text that contain more than two syllables.
        hence we compare it with our words."""
        #complex_words = self.column_with_words.apply(lambda x : len([word for word in x if self.syllables_count(word) > 2]))
        ans = self.complex_words.fillna(0) 
        
        return ans
    def word_counts(self):
        """This function is used to calculate number of words in each document axcept stopwords found in nltk library"""
        stopwords_nltk = np.array(list(set(stopwords.words('english'))))
        words = self.column_with_words.apply(lambda x : len([word for word in x if word not in stopwords_nltk]))
        ans =  words
        ans = ans.fillna(0)
        return ans

    def uncertainty_words_count(self):
        uncertainty_dict = pd.read_excel(r'D:\nlp_internship\Data Science\uncertainty_dictionary.xlsx')
        uncertainty_dict = uncertainty_dict['Word'].str.lower().to_list()
        words = self.column_with_words.apply(lambda x : len([word for word in x if word  in uncertainty_dict]))
        return words
    def constraning_words_count(self):
        constrain_dict = pd.read_excel(r'D:\nlp_internship\Data Science\constraining_dictionary.xlsx')
        constrain_dict = constrain_dict['Word'].str.lower().to_list()
        words = self.column_with_words.apply(lambda x : len([word for word in x if word  in constrain_dict]))
        return words
    def positive_words_proportion(self):
        #words = self.column_with_words.apply(lambda x : len([word for word in x]))
        ans = self.positive_words_column / self.words
        ans = ans.fillna(0)
        return ans
    def negative_words_proportion(self):
        #words = self.column_with_words.apply(lambda x : len([word for word in x]))
        ans = self.negative_words_column / self.words
        ans = ans.fillna(0)
        return ans
    def uncertaninty_words_proportion(self):
        #words = self.column_with_words.apply(lambda x : len([word for word in x]))
        ans = self.uncertainty_words_count() / self.words
        ans = ans.fillna(0)
        return ans
    def constraining_words_proportion(self):
        #words = self.column_with_words.apply(lambda x : len([word for word in x]))
        ans = self.constraning_words_count() / self.words
        ans = ans.fillna(0)
        return ans


In [36]:
final_dataframe.columns

Index(['CIK', 'CONAME', 'FYRMO', 'FDATE', 'FORM', 'SECFNAME', 'matches_mda',
       'matches_qlty_qnt_disc', 'matches_rsk_fct', 'mda_bulk', 'qltqnt_bulk',
       'rskfct_bulk'],
      dtype='object')

In [ ]:
class text_analysis_vectorized:
    def __init__(self,column_with_words,column_with_sentences):
        self.column_with_words = column_with_words # initializing the column which contaions words.
        self.column_with_sentences = column_with_sentences # initializing the words which contains sentence tokens.
        self.words = self.column_with_words.apply(lambda x : len([word for word in x])) # initialing the variable wors count
        self.complex_words = self.column_with_words.apply(lambda x : len([word for word in x if syllables.estimate(word) > 2])) # initializing the variable complex words
        self.positive_words_column = self.column_with_words.apply(lambda x : len([word for word in x if word in np.array(positive_dict)])) # initializing the column number of positive words
        self.negative_words_column = self.column_with_words.apply(lambda x : len([word for word in x if word in np.array(negative_dict)])) # initializing the column number of positive words
    def positive_score(self):
        '''This score is calculated by assigning the value of +1 for each word if 
        found in the Positive Dictionary and then adding up all the values. '''
        return self.positive_words_column
        #return self.column_with_words.apply(lambda x : len([word for word in x if word in positive_dict]))
    def negative_score(self):
        '''This score is calculated by assigning the value of +1 for each word if 
        found in the Negative Dictionary and then adding up all the values.'''
        return self.negative_words_column
        #return self.column_with_words.apply(lambda x : len([word for word in x if word in negative_dict]))
    def polarity_score(self):
        """This is the score that determines if a given text is positive or negative in nature. It is calculated by using the formula: 
            Polarity Score = (Positive Score – Negative Score)/ ((Positive Score + Negative Score) + 0.000001)
        """
        val1 = (self.positive_words_column - self.negative_words_column)
        val2 = (self.positive_words_column + self.negative_words_column + 0.00001)
        return val1 / val2
    def average_sentence_length(self):
        """
        Average Sentence Length = the number of words / the number of sentences
        we use sent_tokenizer liabrary and we replace null values with 0

        """
        #words = self.column_with_words.apply(lambda x : len([word for word in x]))
        sentences = self.column_with_sentences.apply(lambda x : len(sent_tokenize(str(x))))
        ans =  self.words / sentences
        ans = ans.fillna(0)
        return ans
    def syllables_count(self,word):
        """This function is used to count the syllables. In this case, we use the cmu dictionary which consists 
        of all the syllables of given words but I found that there are some words which are not defined in the 
        cmu dictionary hence I used the algorithm to calculate the number of syllables."""
        phoneme_dictonary = dict(cmudict.entries())
        count = 0
        vowels = 'aeiouy'
        word = word.lower()
        if word[0] in vowels:
            count += 1
        for i in range(1,len(word)):
            if word[i] in vowels and word[i - 1] not in vowels:
                count += 1
        if word[-1] == 'e':
            count -= 1
        if word[-2:] == 'le':
            count += 1
        if count == 0:
            count += 1
        # comparing the values with cmudict
        try:
            second_count = 0
            for val in phoneme_dictonary[word.lower()]:
                for val1 in val:
                    if val1[-1].isdigit():
                        second_count += len(val1)
            return second_count
        except KeyError: # this is in case the values are not found in the cmu dict
            return count
    def complex_words_proportion(self):
        """Percentage of Complex words = the number of complex words / the number of words """
        #complex_words = self.column_with_words.apply(lambda x : len([word for word in x if self.syllables_count(word) > 2]))
        #words = self.column_with_words.apply(lambda x : len([word for word in x]))
        ans = self.complex_words / self.words
        ans = ans.fillna(0)
        return ans

    def fog_index(self):
        """Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)
        The Gunning Fog Index gives the number of years of education that your reader 
        hypothetically needs to understand the paragraph or text. The Gunning Fog Index 
        formula implies that short sentences written in plain English achieve a better 
        score than long sentences written in complicated language."""
        return 0.4 * (self.average_sentence_length() + self.complex_words)

    def complex_words_count(self):
        """ Complex words are words in the text that contain more than two syllables.
        hence we compare it with our words."""
        #complex_words = self.column_with_words.apply(lambda x : len([word for word in x if self.syllables_count(word) > 2]))
        ans = self.complex_words.fillna(0) 
        
        return ans
    def word_counts(self):
        """This function is used to calculate number of words in each document axcept stopwords found in nltk library"""
        stopwords_nltk = np.array(list(set(stopwords.words('english'))))
        words = self.column_with_words.apply(lambda x : len([word for word in x if word not in stopwords_nltk]))
        ans =  words
        ans = ans.fillna(0)
        return ans

    def uncertainty_words_count(self):
        uncertainty_dict = pd.read_excel(r'D:\nlp_internship\Data Science\uncertainty_dictionary.xlsx')
        uncertainty_dict = uncertainty_dict['Word'].str.lower().to_list()
        words = self.column_with_words.apply(lambda x : len([word for word in x if word  in uncertainty_dict]))
        return words
    def constraning_words_count(self):
        constrain_dict = pd.read_excel(r'D:\nlp_internship\Data Science\constraining_dictionary.xlsx')
        constrain_dict = constrain_dict['Word'].str.lower().to_list()
        words = self.column_with_words.apply(lambda x : len([word for word in x if word  in constrain_dict]))
        return words
    def positive_words_proportion(self):
        #words = self.column_with_words.apply(lambda x : len([word for word in x]))
        ans = self.positive_words_column / self.words
        ans = ans.fillna(0)
        return ans
    def negative_words_proportion(self):
        #words = self.column_with_words.apply(lambda x : len([word for word in x]))
        ans = self.negative_words_column / self.words
        ans = ans.fillna(0)
        return ans
    def uncertaninty_words_proportion(self):
        #words = self.column_with_words.apply(lambda x : len([word for word in x]))
        ans = self.uncertainty_words_count() / self.words
        ans = ans.fillna(0)
        return ans
    def constraining_words_proportion(self):
        #words = self.column_with_words.apply(lambda x : len([word for word in x]))
        ans = self.constraning_words_count() / self.words
        ans = ans.fillna(0)
        return ans


In [ ]:
# defining the mda object and giving it the concerned values.

In [40]:
mda = text_analysis(final_dataframe['mda_bulk'],final_dataframe['matches_mda']) 

In [41]:
MDA = pd.DataFrame() #creating mda dataframe
MDA['mda_positive_score'] = mda.positive_score()
MDA['mda_negative_score'] = mda.negative_score()
MDA['mda_polarity_score'] = mda.polarity_score()
MDA['mda_average_sentence_length'] = mda.average_sentence_length()
MDA['mda_percenmdage_of_complex_words'] = mda.complex_words_proportion()
MDA['mda_fog_index'] = mda.fog_index()
MDA['mda_complex_word_count'] = mda.complex_words_count()
MDA['mda_word_count'] = mda.word_counts()
MDA['mda_uncertainty_score'] = mda.uncertainty_words_count()
MDA['mda_constraining_score'] = mda.constraning_words_count()
MDA['mda_positive_word_proportion'] = mda.positive_words_proportion()
MDA['mda_negative_word_proportion'] = mda.negative_words_proportion()
MDA['mda_uncertainty_word_proportion'] = mda.uncertaninty_words_proportion()
MDA['mda_constraining_word_proportion'] = mda.constraining_words_proportion()
 

In [42]:
MDA.to_csv('mda.csv')

In [43]:
qqdmr = text_analysis(final_dataframe['qltqnt_bulk'],final_dataframe['matches_qlty_qnt_disc'])
QQDMR = pd.DataFrame()
QQDMR['qqdmr_positive_score'] = qqdmr.positive_score()
QQDMR['qqdmr_negative_score'] = qqdmr.negative_score()
QQDMR['qqdmr_polarity_score'] = qqdmr.polarity_score()
QQDMR['qqdmr_average_sentence_length'] = qqdmr.average_sentence_length()
QQDMR['qqdmr_percenqqdmrge_of_complex_words'] = qqdmr.complex_words_proportion()
QQDMR['qqdmr_fog_index'] = qqdmr.fog_index()
QQDMR['qqdmr_complex_word_count'] = qqdmr.complex_words_count()
QQDMR['qqdmr_word_count'] = qqdmr.word_counts()
QQDMR['qqdmr_uncertainty_score'] = qqdmr.uncertainty_words_count()
QQDMR['qqdmr_constraining_score'] = qqdmr.constraning_words_count()
QQDMR['qqdmr_positive_word_proportion'] = qqdmr.positive_words_proportion()
QQDMR['qqdmr_negative_word_proportion'] = qqdmr.negative_words_proportion()
QQDMR['qqdmr_uncertainty_word_proportion'] = qqdmr.uncertaninty_words_proportion()
QQDMR['qqdmr_constraining_word_proportion'] = qqdmr.constraining_words_proportion()

QQDMR.to_csv('qqdmr.csv')

In [45]:
rf = text_analysis(final_dataframe['rskfct_bulk'],final_dataframe['matches_rsk_fct'])
RF = pd.DataFrame()
RF['rf_positive_score'] = rf.positive_score()
RF['rf_negative_score'] = rf.negative_score()
RF['rf_polarity_score'] = rf.polarity_score()
RF['rf_average_sentence_length'] = rf.average_sentence_length()
RF['rf_percenrfge_of_complex_words'] = rf.complex_words_proportion()
RF['rf_fog_index'] = rf.fog_index()
RF['rf_complex_word_count'] = rf.complex_words_count()
RF['rf_word_count'] = rf.word_counts()
RF['rf_uncertainty_score'] = rf.uncertainty_words_count()
RF['rf_constraining_score'] = rf.constraning_words_count()
RF['rf_positive_word_proportion'] = rf.positive_words_proportion()
RF['rf_negative_word_proportion'] = rf.negative_words_proportion()
RF['rf_uncertainty_word_proportion'] = rf.uncertaninty_words_proportion()
RF['rf_constraining_word_proportion'] = rf.constraining_words_proportion()
RF.to_csv('rf.csv')


In [51]:
pd.concat([MDA,QQDMR,RF],axis = 1).to_csv('highly_efficient.csv')

,mda_positive_score,mda_negative_score,mda_polarity_score,mda_average_sentence_length,mda_percenmdage_of_complex_words,mda_fog_index,mda_complex_word_count,mda_word_count,mda_uncertainty_score,mda_constraining_score,mda_positive_word_proportion,mda_negative_word_proportion,mda_uncertainty_word_proportion,mda_constraining_word_proportion
0,17,62,-0.569620,2011.0,0.521134,1223.6,1048,1915,23,10,0.008454,0.030830,0.011437,0.004973
1,9,46,-0.672727,1570.0,0.505096,945.2,793,1504,50,2,0.005732,0.029299,0.031847,0.001274
2,0,0,0.000000,0.0,0.000000,0.0,0,0,0,0,0.000000,0.000000,0.000000,0.000000
3,39,120,-0.509434,3812.0,0.548006,2360.4,2089,3624,61,39,0.010231,0.031480,0.016002,0.010231
4,0,0,0.000000,0.0,0.000000,0.0,0,0,0,0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,0,0,0.000000,0.0,0.000000,0.0,0,0,0,0,0.000000,0.000000,0.000000,0.000000
148,0,0,0.000000,0.0,0.000000,0.0,0,0,0,0,0.000000,0.000000,0.000000,0.000000
149,0,0,0.000000,0.0,0.000000,0.0,0,0,0,0,0.000000,0.000000,0.000000,0.000000
150,0,0,0.000000,0.0,0.000000,0.0,0,0,0,0,0.000000,0.000000,0.000000,0.000000


In [54]:
# finding constraining words for the whole report.

In [58]:
complete_array = []
count = 0
for hyperlink in df['SECFNAME']:
    text = urllib.request.urlopen(hyperlink).read().decode('utf-8')
    count += 1
    complete_array.append(preliminary_cleaner(text))
    print(count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152


In [77]:
whole_document_dataframe = pd.DataFrame()
stopwords_nltk = np.array(list(set(stopwords.words('english'))))
whole_document_dataframe['constraining'] = complete_array

In [78]:
whole_document_dataframe = cleaning_stopwords(whole_document_dataframe['constraining'])

In [82]:
arr = pd.DataFrame(whole_document_dataframe)

In [92]:
union_array = arr['constraining'].apply(lambda x : np.union1d(np.array(x),stopwords_nltk))
uncertainty_dict = pd.read_excel(r'D:\nlp_internship\Data Science\uncertainty_dictionary.xlsx')
uncertainty_dict = np.array(uncertainty_dict['Word'].str.lower().to_list())

In [94]:
ans = union_array.apply(lambda x : len(np.intersect1d(x,uncertainty_dict)))